# 09. 비동기

내용이 방대해 네트워크, 스트림, 쓰레드 등은 다루지 않고 필요한 내용만 다루었다. 

본 장에서는 고수준 API와 저수준 API 모두 소개했지만 현재 당장은 고수준 API만 알아도 무방할 듯하다.

<br><br><br>

## - asyncio

`async/await` 구문을 사용하여 동시성 코드를 작성하는 라이브러리

`고수준 API`와 `저수준 API`를 지원한다.

 -  [고수준 API Reference](https://docs.python.org/ko/3/library/asyncio-api-index.html)

 -  [저수준 API Reference](https://docs.python.org/ko/3/library/asyncio-llapi-index.html)

```python
import asyncio
```

<br><br><br>

## - 고수준 API

### * async/await 개요

코루틴을 선언할 때는 함수(def) 옆 키워드로 `async`를 붙이면 된다. 그리고 `await`는 `async`로 정의된 함수 안에서만 사용이 가능하며 `await`는 코루틴을 실행하고 끝날 때까지 기다린다.

### * 코루틴 함수 and 코루틴 객체

`코루틴 함수`는 `async def`로 선언된 함수를 이야기하고 `코루틴 객체`는 코루틴 함수를 호출하여 반환된 코루틴을 이야기한다.

### * awaitable 객체

`await` 표현식을 통해 사용할 수 있으면 어웨이터블 객체라고 말한다.

 - 코루틴(Coroutine)
 - 태스크(Task)
 - 퓨처(Futrue)

### * asyncio.run(coro, *, debug=False)

동기 코드에서 비동기 코드로 진입하는 부분이다. 선언된 코루틴을 `coro` 위치에 인수로서 대입하면 된다.

내부적으로 이벤트 루프(even_loop)를 만들고 코루틴(coroutine)을 실행한 다음 이벤트 루프를 닫는다(close).

### * asyncio.sleep(delay, result=None, *, loop=None)

delay 동안 Blocking한다. 즉, 잠깐 멈춘다.

result가 있으면 완료시 호출자에게 반환한다.

<hr>

```python
import asyncio

async def main():
    print('hello')
    await asyncio.sleep(1)
    print('world')

asyncio.run(main());
```

 - 결과
 
```
hello
world
```

<hr>

또 다른 코드를 실행해보자.

<hr>

```python
import asyncio
import time

async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)

async def main():
    
    # the '%X' means Locale’s appropriate time representation.
    print(f"started at {time.strftime('%X')}") 

    await say_after(3, 'hello')
    await say_after(4, 'world')

    print(f"finished at {time.strftime('%X')}")

asyncio.run(main())
```

 - 결과
 
```
started at 00:56:52
hello
world
finished at 00:56:59
```

<hr>

위 코드가 모두 마치기까지 약 7초가 걸렸다. 이유는 먼저 `say_after(3, 'hello')` 코루틴을 실행하고 약 3초간 기다렸다가 마무리된 후 `say_after(4, 'world')` 코루틴에서 4초간 기다렸기 때문이다.

`time.strftime()` 함수에 대해 궁금하다면 공식 문서를 참고하자. [click](https://docs.python.org/3.8/library/time.html#time.strftime)

때문에 각 코루틴의 결과값에 서로 의존적이지 않다면 각각의 코루틴이 모두 따로 동시에 실행되도록 하는 솔루션이 필요하다.

### * asyncio.create_task(coro, *, name=None)

흐름을 분기하는 코드이며, 새로운 micro-thread 생성한다.

전달 받은 코루틴을 Task로 감싸고 실행을 예약한다.

그리고 그러한 Task 객체를 반환한다.

name이 None이 아니면 내부적으로 `Task.set_name()`을 사용해 Task 이름을 설정한다.



<hr>

```python
import asyncio
import time

async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)

async def main():
    coro1 =  say_after(3, 'hello')
    coro2 =  say_after(4, 'world')

    task1 = asyncio.create_task(coro1)
    task2 = asyncio.create_task(coro2)

    print(f"started at {time.strftime('%X')}")

    await task1
    await task2

    print(f"finished at {time.strftime('%X')}")

asyncio.run(main())
```

 - 결과
 
```
started at 01:09:12
hello
world
finished at 01:09:16
```

<hr>

위에 결과를 보면 알 수 있듯이 각 코루틴이 Task 객체로 감싸져서 나왔고 코루틴을 실행했던 것처럼 `await` 키워드를 통해 비동기적인 실행을 하였다. 그러나 각 코루틴이 끝날 때까지 기다렸다가 다음 코루틴 작업으로 넘어가는 것이 아니라 동시에 실행한 것을 볼 수 있다.

또다른 예제를 살펴보자.

<hr>

```python
import asyncio

async def saying(interval, what, how_many):
    while how_many > 0:
        await asyncio.sleep(interval)
        print(what)
        how_many -= 1

async def main():
    coro1 =  saying(1, 'hello', 5)
    coro2 =  saying(2, 'world', 3)

    await coro1
    await coro2

asyncio.run(main())
```

 - 결과
 
```
hello
hello
hello
hello
hello
world
world
world
```

<hr>

본 예제도 마찬가지로 코루틴 프로그래밍을 통해 비동기적인 프로그램 실행을 노렸지만 결과적으로는 마치 동기적으로 실행되어버린 케이스다.

물론 `create_task()` 함수를 통해 코루틴을 Task 객체로 감싸서 동시에 처리할 수 있다.

<hr>

```python
import asyncio

async def saying(interval, what, how_many):
    while how_many > 0:
        await asyncio.sleep(interval)
        print(what)
        how_many -= 1

async def main():
    coro1 =  saying(1, 'hello', 5)
    coro2 =  saying(2, 'world', 3)
    coro3 =  saying(3, 'python', 2)

    task1 = asyncio.create_task(coro1)
    task2 = asyncio.create_task(coro2)
    task3 = asyncio.create_task(coro3)

    await task1
    await task2
    await task3

asyncio.run(main())
```

 - 결과
 
```
hello
world
hello
python
hello
world
hello
hello
world
python
```

<hr>

그리고 `gather()` 함수로도 여러가지를 동시에 예약하고 처리할 수도 있다. 

### * asyncio.gather(*aws, loop=None, return_exceptions=False)

aws 파라미터에 들어간 코루틴 객체들을 자동으로 태스크로 예약시키고 동시에 실행한다. 처리 결과값들은 선언된 순서와 일치한다.

<hr>

```python
import asyncio

async def saying(interval, what, how_many):
    while how_many > 0:
        await asyncio.sleep(interval)
        print(what)
        how_many -= 1
    return what

async def main():
    coro1 =  saying(1, 'hello', 5)
    coro2 =  saying(2, 'world', 3)
    coro3 =  saying(3, 'python', 2)

    coro_result_list = await asyncio.gather(
        coro1, coro2, coro3
    )
    print(coro_result_list)

asyncio.run(main())
```

 - 결과
 
```
hello
world
hello
python
hello
hello
world
hello
python
world
['hello', 'world', 'python']
```

<hr>

<br><br><br>

## - 저수준 API

asyncio 모듈의 고수준 API는 응용 프로그램 개발자를 위한 것이었다면, 저수준 API는 라이브러리와 프레임워크 개발자를 위한 것이다.

아래 코드는 이미 위에서 경험한 고수준 API를 이용한 프로그램이다.

<hr>

```python
import asyncio

async def main():
    print('hello')
    await asyncio.sleep(1)
    print('world')

asyncio.run(main());
```

 - 결과
 
```
hello
world
```

<hr>

이를 저수준 API 형식으로 바꾸면 다음과 같다.

<hr>

```python
import asyncio

async def main():
    print('hello')
    await asyncio.sleep(1)
    print('world')

loop = asyncio.get_event_loop()
loop.run_until_complete(main())
loop.close()
```

 - 결과
 
```
hello
world
```

<hr>

왜 `asyncio.run()`와 똑같이 동작할까? 이 함수에 대한 설명을 다시 살펴보자. 이 안에 답이 있다.

### * asyncio.run()

동기 코드에서 비동기 코드로 진입하는 부분이다. 선언된 코루틴을 `coro` 위치에 인수로서 대입하면 된다.

내부적으로 이벤트 루프(even_loop)를 만들고 코루틴(coroutine)을 실행한 다음 이벤트 루프를 닫는다(close).

여기서 주목할 부분은 다음과 같다.

```
...
...
내부적으로 이벤트 루프(even_loop)를 만들고 코루틴(coroutine)을 실행한 다음 이벤트 루프를 닫는다(close).
...
...
```

이 말은 아래 코드를 말하는 것이다.

```python
loop = asyncio.get_event_loop()
loop.run_until_complete(main())
loop.close()
```

### * asyncio.get_event_loop()

현재 사용중인 이벤트 루프를 가져온다. 

파이썬 공식 문서에서는 이러한 저수준의 함수를 사용하여 수동으로 이벤트 루프를 생성하고 닫는 대신 그냥 `asyncio.run()` 함수를 사용하는 것을 고려하라고 한다.

### * loop.run_until_complete(future)
 Future의 인스턴스가 완료가 될 때까지 실행하고 만일 Future 객체가 아니라 코루틴 객체가 들어오면 TasK 객체로 실행되도록 묵시적으로 예약된다.

### * loop.close()

이벤트 루트를 닫는다.

### * Future 객체

Future 객체는 저수준 콜백 기반 코드와 고수준 async/await 코드 간에 다리를 놓는데 사용된다. 또한 비동기 연산의 최종 결과를 나타내는 특별한 저수준 awaitable 객체이다.

콜백 기반 코드를 async/await와 함께 사용하려면 asyncio의 Future 객체가 필요하다. 일반적으로 응용 프로그램 수준에서는 Future 객체를 만들 필요는 없다.

간단하게 어떤식으로 사용되어지는지 코드만 보도록 하자.

단순하게 응용 프로그램을 구현할 것이면 고수준 API를 주로 사용할 것이고 저수준 API가 필요하다면 그 때 문서를 보며 파악하면 된다.

<hr>

```python
import asyncio

def function_that_returns_a_future_object():
    print("future object function~!")
    loop = asyncio.get_event_loop()
    loop.call_soon(lambda : print("hello world"))
    future = loop.create_future()
    future.set_result("success")
    return future

async def main():
    result = await function_that_returns_a_future_object()
    print(result)
    print("AAAAAAAA")
    print("BBBBBBBB")
    print("CCCCCCCC")
    
# loop = asyncio.get_event_loop()
# loop.run_until_complete(main())
# loop.close()
asyncio.run(main())
```

 - 결과
 
```
future object function~!
success
AAAAAAAA
BBBBBBBB
CCCCCCCC
hello world
```

<hr>

<br><br><br>

## - 참고사항

## * 제너레이터 코루틴

제너레이터 기반 코루틴은 async/await 문법 전에 나왔다.

제너레이터 기반 코루틴이란 `yield from` 표현식을 사용하여 구현하는 파이썬 제너레이터이다.

파이썬 코루틴 관련 인터넷 자료 및 강의들을 보면 파이썬 제너레이터에 대한 기초 강의 후 제너레이터 기반 코루틴을 알려주는 데가 많다.

그러나 버전 3.8부터 이러한 방식은 Deprecated 되었다. 즉, 제너레이터 기반 코루틴에 대한 지원은 폐지되었다. 그리고 버전 3.10 때 삭제될 예정이다.

파이썬 공식 문서는 제너레이터 코루틴 대신 `async def`를 사용하기를 권장하고 있다.

 - Use Example
 
```python
@asyncio.coroutine
def old_style_coroutine():
    yield from asyncio.sleep(1)
async def main():
    await old_style_coroutine()
```

`@asyncio.coroutine` 데코레이터 사용이 강제되지는 않는다

### * Blocking vs. Non-blocking

 - 입출력 처리를 완료될 때까지 기다릴 것인지 혹은 시작만 해두고 다음 작업을 계속 진행할 것인지 여부
 - 후속 작업이 있는 경우 Polling이나 Callback을 사용

### * Sync vs. Async Programming

 - 프로그램의 주 실행 흐름을 멈추지 않고 진행할 수 있는가의 여부
 - 코드의 실행 결과 처리및 활용을 별도의 채널에 맡겨둔 뒤 결과를 기다리지 않고 다음 코드를 실행하는 방식의 프로그래밍
 - Futrue, Promise, Coroutine 등

### * 현재 유명한 Async Framework
 - Fast API
 - Vibora
 - AIOHTTP
 - Sanic
 - Tornado (async/await 지원 이전에 많이 사용, 요즘은 잘 사용 안함)

<br>
<br>
<br>
<br>
<br>
<br>
<hr>
<br>
<br>
<br>
<br>
<br>
<br>